## 보쉬 생산 라인 데이터 분석

이 노트북에서는 보쉬 생산 라인 데이터를 사용하여 어떤 부품이 고장날지 예측하는 방법을 배워보겠습니다. 중학생도 이해할 수 있도록 쉽게 설명할게요!

### 1. 라이브러리 불러오기

먼저 데이터 분석에 필요한 도구들(라이브러리)을 불러옵니다. `pandas`는 데이터를 표 형태로 다루게 해주고, `matplotlib`과 `seaborn`은 데이터를 그래프로 예쁘게 보여주는 도구입니다.

In [3]:
# pandas 라이브러리를 pd라는 이름으로 불러옵니다.
import pandas as pd
# matplotlib.pyplot 라이브러리를 plt라는 이름으로 불러옵니다.
import matplotlib.pyplot as plt
# seaborn 라이브러리를 sns라는 이름으로 불러옵니다.
import seaborn as sns

# 그래프 스타일을 정합니다.
sns.set(style="whitegrid")

### 2. 데이터 불러오기

이제 데이터를 불러올 차례입니다. 데이터 파일은 매우 크기 때문에, 여기서는 처음 100,000개의 행만 샘플로 사용하겠습니다. `train_numeric.csv` 파일이 `data` 폴더 안에 있어야 합니다.

In [4]:
# train_numeric.csv 파일에서 처음 100,000개의 행만 읽어옵니다.
# 파일 경로가 올바른지 확인해주세요.
data = pd.read_csv('data/train_numeric.csv', nrows=100000)

# 데이터의 첫 5줄을 출력하여 잘 불러왔는지 확인합니다.
print("데이터 샘플:")
print(data.head())

FileNotFoundError: [Errno 2] No such file or directory: 'data/train_numeric.csv'

### 3. 데이터 탐색하기 (EDA)

데이터를 자세히 들여다보며 어떤 정보가 들어있는지 알아보겠습니다.

In [ ]:
# 데이터가 몇 개의 행과 열로 이루어져 있는지 확인합니다.
print(f'데이터는 {data.shape[0]}개의 행과 {data.shape[1]}개의 열로 이루어져 있습니다.')

In [ ]:
# 'Response' 열에 어떤 값들이 있는지, 그리고 각각 몇 개씩 있는지 확인합니다.
# 0은 정상, 1은 불량을 의미합니다.
print('
불량 여부 (Response) 분포:')
print(data['Response'].value_counts())

In [ ]:
# 'Response' 열의 분포를 막대 그래프로 그려봅니다.
plt.figure(figsize=(6, 4))
sns.countplot(x='Response', data=data)
plt.title('불량 여부 (Response) 분포')
plt.xlabel('0: 정상, 1: 불량')
plt.ylabel('개수')
plt.show()

### 4. 데이터 전처리

모델이 학습하기 좋도록 데이터를 정리하는 과정입니다. 여기서는 비어있는 값(결측치)을 채워보겠습니다.

In [ ]:
# 각 열에 비어있는 값이 몇 개나 있는지 확인합니다.
missing_values = data.isnull().sum()
print("각 열의 결측치 개수:")
print(missing_values[missing_values > 0])

# 비어있는 값을 각 열의 평균값으로 채웁니다.
data = data.fillna(data.mean())

# 결측치를 채운 후 다시 확인합니다.
print('
결측치 처리 후, 다시 확인:')
print(data.isnull().sum().sum())

### 5. 모델 학습을 위한 데이터 준비

이제 모델을 학습시킬 차례입니다. 예측하고 싶은 값인 `Response`를 제외한 나머지 데이터를 문제(X)로, `Response`를 정답(y)으로 나눕니다.

In [ ]:
# 문제(X)와 정답(y)으로 데이터를 나눕니다.
X = data.drop(['Id', 'Response'], axis=1)
y = data['Response']

### 6. 학습 데이터와 테스트 데이터 분리

모델을 학습시킬 데이터와 모델의 성능을 평가할 데이터를 나눕니다. 이렇게 하면 모델이 새로운 데이터에 대해서도 잘 예측하는지 확인할 수 있습니다.

In [ ]:
# scikit-learn 라이브러리에서 필요한 도구를 불러옵니다.
from sklearn.model_selection import train_test_split

# 데이터를 학습용과 테스트용으로 8:2 비율로 나눕니다.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### 7. 결정 트리 모델 학습

이제 결정 트리 모델을 만들고 학습시킵니다. 결정 트리는 스무고개처럼 질문을 통해 정답을 찾아가는 모델입니다.

In [ ]:
# scikit-learn 라이브러리에서 결정 트리 모델을 불러옵니다.
from sklearn.tree import DecisionTreeClassifier

# 결정 트리 모델을 만듭니다.
model = DecisionTreeClassifier(max_depth=5, random_state=42)

# 모델을 학습시킵니다.
model.fit(X_train, y_train)

### 8. 모델 평가

학습된 모델이 얼마나 정확하게 예측하는지 확인합니다.

In [ ]:
# scikit-learn 라이브러리에서 정확도 계산 도구를 불러옵니다.
from sklearn.metrics import accuracy_score

# 테스트 데이터로 예측을 수행합니다.
y_pred = model.predict(X_test)

# 실제 값과 예측 값을 비교하여 정확도를 계산합니다.
accuracy = accuracy_score(y_test, y_pred)
print(f'모델 정확도: {accuracy * 100:.2f}%')

### 9. 결정 트리 시각화

모델이 어떻게 예측하는지 직접 눈으로 확인해봅시다. (그래프가 복잡할 수 있습니다)

In [ ]:
# scikit-learn 라이브러리에서 트리 시각화 도구를 불러옵니다.
from sklearn.tree import plot_tree

# 그래프 크기를 정합니다.
plt.figure(figsize=(20, 10))

# 결정 트리를 그립니다.
plot_tree(model, filled=True, feature_names=X.columns, class_names=['정상', '불량'], max_depth=2, fontsize=10)

# 그래프를 보여줍니다.
plt.show()